In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
def get_all(_id):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get('https://leader-id.ru/specials/?special_models_Special_page=%s' % _id, headers=headers)
    root = BeautifulSoup(html.content.decode(), "html.parser")
    elements = root.findAll('div', {'class': 'events-item'})
    urls = []
    headers = []
    
    for el in elements:
        urls.append(el.findAll('a')[1]['href'])
        headers.append(el.findAll('a')[1].text)
        
    return urls,headers

In [5]:
urls = set()
headers = set()
old_len = -1
page_num = 1

while old_len != len(urls):
    old_len = len(urls)
    
    el = get_all(page_num)
    for a,b in zip(el[0], el[1]):
        urls.add(a)
        headers.add(b)
    
    page_num += 1

In [53]:
normal_data = []

for j, k in zip(*data):
    normal_data.append([j,k])

In [54]:
normal_data[0]

['/specials/national-technology-initiative/',
 'Национальная технологическая инициатива']

In [55]:
data = pd.DataFrame(normal_data, columns=['url', 'header'])

In [56]:
data['url'] = list(map(lambda x: "https://leader-id.ru%s" % x, data['url']))

In [57]:
data.head()

,url,header
0,https://leader-id.ru/specials/national-technol...,Национальная технологическая инициатива
1,https://leader-id.ru/specials/itstartup/,Финал Предакселератора GenerationS
2,https://leader-id.ru/specials/foresighttrip201...,Форсайт-флот 2016
3,https://leader-id.ru/specials/lecture_forum2016/,Лекторий Форума стратегических инициатив
4,https://leader-id.ru/specials/investclimat/,Инвестклимат.рф


In [75]:
def get_text(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    html = requests.get(url, headers=headers)
    root = BeautifulSoup(html.content.decode(), "html.parser")
    
    if html.status_code == 200:
        answer = root.findAll('div', {'class': 'b-search'})
        
        if len(answer) > 0:
            return answer[0].text
        else:
            print(url)
            return ""
    else:
        return ""

In [76]:
data['text'] = list(map(get_text, data['url']))

https://leader-id.ru/specials/lecture_forum2016/
https://leader-id.ru/specials/lecture_forum2016/
https://leader-id.ru/specials/vdnh_forum_2016/


In [79]:
data = data[data['text'].str.len() > 0]

In [80]:
len(data)

41

In [82]:
data.to_csv('leader-id_special.csv')